# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f25b64d2b80>
├── 'a' --> tensor([[ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506]])
└── 'x' --> <FastTreeValue 0x7f25b64d2190>
    └── 'c' --> tensor([[ 0.1605,  0.0481, -1.2596,  0.9228],
                        [ 0.1301, -0.2952, -1.0788, -0.0202],
                        [ 0.5730, -0.3408, -0.3105,  0.7983]])

In [4]:
t.a

tensor([[ 0.8073, -0.3800,  0.1794],
        [ 0.7944,  1.5094, -1.2506]])

In [5]:
%timeit t.a

86.6 ns ± 0.44 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f25b64d2b80>
├── 'a' --> tensor([[ 0.3499,  0.0298,  0.5287],
│                   [-0.1747,  1.6837, -0.1686]])
└── 'x' --> <FastTreeValue 0x7f25b64d2190>
    └── 'c' --> tensor([[ 0.1605,  0.0481, -1.2596,  0.9228],
                        [ 0.1301, -0.2952, -1.0788, -0.0202],
                        [ 0.5730, -0.3408, -0.3105,  0.7983]])

In [7]:
%timeit t.a = new_value

89.7 ns ± 0.47 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506]]),
    x: Batch(
           c: tensor([[ 0.1605,  0.0481, -1.2596,  0.9228],
                      [ 0.1301, -0.2952, -1.0788, -0.0202],
                      [ 0.5730, -0.3408, -0.3105,  0.7983]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8073, -0.3800,  0.1794],
        [ 0.7944,  1.5094, -1.2506]])

In [11]:
%timeit b.a

75.9 ns ± 0.271 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7542,  2.4856,  1.3376],
               [ 0.1636,  1.7298, -0.2545]]),
    x: Batch(
           c: tensor([[ 0.1605,  0.0481, -1.2596,  0.9228],
                      [ 0.1301, -0.2952, -1.0788, -0.0202],
                      [ 0.5730, -0.3408, -0.3105,  0.7983]]),
       ),
)

In [13]:
%timeit b.a = new_value

638 ns ± 4.92 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 59.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

165 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

163 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f25b4447e80>
├── 'a' --> tensor([[[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]],
│           
│                   [[ 0.8073, -0.3800,  0.1794],
│                    [ 0.7944,  1.5094, -1.2506]]])
└── 'x' --> <FastTreeValue 0x7f24ee7e9cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.2 µs ± 809 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f24dbcbfca0>
├── 'a' --> tensor([[ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506],
│                   [ 0.8073, -0.3800,  0.1794],
│                   [ 0.7944,  1.5094, -1.2506]])
└── 'x' --> <FastTreeValue 0x7f24dbcbfc10>
    └── 'c' --> tensor([[ 0.1605,  0.0481, -1.2596,  0.9228],
                        [ 0.1301, -0.2952, -1.0788, -0.0202],
                 

In [23]:
%timeit t_cat(trees)

35.4 µs ± 671 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

133 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]],
       
               [[ 0.8073, -0.3800,  0.1794],
                [ 0.7944,  1.5094, -1.2506]]]),
    x: Batch(
           c: tensor([[[ 0.1605,  0.0481, -1.2596,  0.9228],
                       [ 0.1301, -0.2952, -1.0788, -0.0202],
                       [ 0.5730, -0.3408, -0.3105,  0.7983]],
         

In [26]:
%timeit Batch.stack(batches)

125 µs ± 9.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506],
               [ 0.8073, -0.3800,  0.1794],
               [ 0.7944,  1.5094, -1.2506]]),
    x: Batch(
           c: tensor([[ 0.1605,  0.0481, -1.2596,  0.9228],
                      [ 0.1301, -0.2952, -1.0788, -0.0202],
                      [ 0.5730, -0.3408, -0.3105,  0.7983],
                      [ 0.1605,  0.0481, -1.2596,  0.9228],
                      [ 0.1301, -

In [28]:
%timeit Batch.cat(batches)

171 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

394 µs ± 7.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
